# 1.3 Explore My Model
# Note: Hardward settings should be: 
+ **GPU model: P100,  connection: 10ge or higher, one GPU per chunk!,  CPU: 4, Memory: 63GB, Chunk: 2**

# Problems that I have met and need to Check:
+ hardware setting in palmetto should be **GPU model: P100,  connection: 10ge or higher, one GPU per chunk!,  CPU: 4, Memory: 63GB, Chunk: 2**.  **The number of GPU should be 1**, otherwise, pytorch will raise Runtime Error: Memory is illegal and can not be accessed.
I have not figure out why this happen. I may do this later
+ In Neural network model, the output should be  avgpooling -> linear(in_fea, 200)-> relu -> linear(200, 1), **we have 2 linear layers after convolution**
+ in reshaping batch samples: **Use permute to transpose sample, NOT view!**， using view() could lead to disorder of data (different axises of data are mixed together!)
+ Use linear layer output + BCEWithlogitLoss  Or  sigmoid output + BCELoss
+ **Use Adam or AdaGrad Optimizer in this experiment, Don't Use SGD!** Since the Loss is too large when SGD and hard to optimize
+ If you see the training accuracy doesn't increase and model doesn't fit in pytorch, **Reset the notebook or Re-check the program! Sometimes the notebook stores some previous data we don't want to see, but the was used in notebook!**

In [1]:
import numpy as np
import pandas as pd
import torch
from torch import optim, nn
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
from dataset import create_train_test_file_list, Person_MealsDataset, balance_data_indices


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Device: ",device,"Device Count: ", torch.cuda.device_count(), "Device Name: ",torch.cuda.get_device_name()  )
    print("Torch version:", torch.__version__)


In [2]:
import sys

### imports
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Turn off TensorFlow logging
import tensorflow.compat.v1 as tf # maintain compatibility with TensorFlow 2.2.0

import keras
# from tensorflow.compat.v1.keras import backend as K # changed for compatibility with TensorFlow 2.2.0
import numpy as np
import pandas as pd
import random
#from sklearn.metrics import classification_report, confusion_matrix
from datetime import datetime
import loadfile
import addons

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape, Activation
from keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D
from keras.utils import np_utils
from keras.regularizers import l1
from keras.models import load_model, save_model

from tensorflow.compat.v1.keras import backend as K
import numpy as np

shimmer_global_mean = [-0.012359981,-0.0051663737,0.011612018,
                        0.05796114,0.1477952,-0.034395125 ]

shimmer_global_stddev = [0.05756385,0.040893298,0.043825723,
                        17.199743,15.311142,21.229317 ]

shimmer_trended_mean = [-0.000002,-0.000002,-0.000000,
                0.058144,0.147621,-0.033260 ]

shimmer_trended_stddev = [0.037592,0.034135,0.032263,
                17.209038,15.321441,21.242532 ]

all_zero_means = [0,0,0,0,0,0]

meanvals = all_zero_means
stdvals = shimmer_trended_stddev


random_seed  = 1000


In [ ]:
tf.disable_eager_execution()

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

In [3]:
ModelNum = 3
EPOCHS = 10
winmin = 1 
stridesec = 1

outfile = sys.stdout

winlength = int(winmin * 60 * 15)
step = int(stridesec * 15)
start_time = datetime.now()
arr = ["echo -n 'PBS: node is '; cat $PBS_NODEFILE",\
      "echo PBS: job identifier is $PBS_JOBID",\
      "echo PBS: job name is $PBS_JOBNAME"]

pathtemp = "../models/ActiModels/M_F_"
modelpath = pathtemp + "{:f}Min.h5".format(winmin)
jsonpath = pathtemp + "{:f}Min.json".format(winmin)

[os.system(cmd) for cmd in arr]
print("*****************************************************************\n", file=outfile, flush=True)
print("Execution Started at " + start_time.strftime("%m/%d/%Y, %H:%M:%S"), file=outfile, flush=True)
print("WindowLength: {:.2f} min ({:d} datum)\tSlide: {:d} ({:d} datum)\tEpochs:{:d}\n".format(winmin, winlength, stridesec, step, EPOCHS), file=outfile, flush=True)


# Load the dataset
meal_data_train = Person_MealsDataset(person_name= "wenkanw", file_name = "train_files", winmin = winmin,stridesec = stridesec)
meal_data_test = Person_MealsDataset(person_name= "wenkanw", file_name = "test_files", winmin = winmin,stridesec = stridesec)
# meal_data_train = Person_MealsDataset(person_name= "wenkanw", file_name = "all_files_list", winmin = winmin,stridesec = stridesec)

print("Data loaded", file=outfile, flush=True)
print("Dataset ready", file=outfile, flush=True)


*****************************************************************

Execution Started at 12/17/2020, 14:26:56
WindowLength: 1.00 min (900 datum)	Slide: 1 (15 datum)	Epochs:10

Loading Dataset ...
Loading File:  ../data/IndividualData/wenkanw-data/10-14-20/10-14-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/10-3-20/10-3-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/10-9-20/10-10-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/10-9-20/10-9-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-14-20/11-14-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-16-20/11-16-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-18-20/11-18-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-21-20/Dinner/Dinner.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-21-20/lunch/lunch.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-22-20/Dinner/Dinner.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-22-20/Lunch/L

In [4]:
unique_elements, counts_elements = np.unique(meal_data_train.labels, return_counts=True)
print(unique_elements)
print(counts_elements)

### Set the seed for randomizing
random.seed(random_seed)

# load
print("Training using every file\n",file=outfile, flush=True)
trainingsamples,traininglabels =  meal_data_train.data_indices, meal_data_train.labels
testsamples,testlabels =  meal_data_test.data_indices, meal_data_test.labels

train_shuffledUnderSampledBalancedIndices = balance_data_indices(traininglabels,mode="under", shuffle=True)
test_shuffledUnderSampledBalancedIndices = balance_data_indices(testlabels,mode="under", shuffle=True)



[0 1]
[333221  20939]
Training using every file



In [ ]:
print("Creating balanced training data array", file=outfile, flush=True)
balancedData,balancedLabels = meal_data_train.get_subset( train_shuffledUnderSampledBalancedIndices)
balancedData_test,balancedLabels_test = meal_data_test.get_subset( test_shuffledUnderSampledBalancedIndices)


# Tensorflow model

In [16]:
print("*****************************************************************\n", file=outfile, flush=True)
print("Training on {:d} samples of length {:d}\n".format(len(shuffledUnderSampledBalancedIndices), len(balancedData[0])), file=outfile, flush=True)


K.get_session().close()
K.set_session(tf.Session())
K.get_session().run(tf.global_variables_initializer())

mcp_save = keras.callbacks.ModelCheckpoint(modelpath, save_best_only=True, monitor='accuracy')

model = Sequential()
model.add(Conv1D(10, 44, strides=2,activation='relu', input_shape=(winlength, 6)))
model.add(Conv1D(10, 20, strides=2, activation='relu',kernel_regularizer=keras.regularizers.l1(0.01)))
model.add(Conv1D(10, 4, strides=2, activation='relu',kernel_regularizer=keras.regularizers.l1(0.01)))
model.add(GlobalAveragePooling1D())
model.add(Dense(200, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
        optimizer='adam', metrics=['accuracy'])

H = model.fit(x=balancedData, y = balancedLabels,
#                validation_data=[undersampledTestData, undersampledTestLabels],
            epochs = EPOCHS, batch_size=256, verbose=1, validation_split=0.2,
            callbacks=[mcp_save]) # removed addons.LossHistory(jsonpath) for compatibility with TensorFlow 2.2.0, needs to be re-added at some point

print("Max value: ", max(H.history['accuracy']), " at epoch", np.argmax(H.history['accuracy']) + 1)
K.get_session().close()

*****************************************************************

Training on 8940 samples of length 900

Train on 36529 samples, validate on 9133 samples
Epoch 1/10
36529/36529 [==============================] - 23s 636us/sample - loss: 1.3206 - accuracy: 0.6670 - val_loss: 0.7854 - val_accuracy: 0.7021
Epoch 2/10
36529/36529 [==============================] - 23s 638us/sample - loss: 0.6816 - accuracy: 0.7116 - val_loss: 0.6520 - val_accuracy: 0.6989
Epoch 3/10
36529/36529 [==============================] - 24s 653us/sample - loss: 0.6115 - accuracy: 0.7235 - val_loss: 0.6021 - val_accuracy: 0.7324
Epoch 4/10
36529/36529 [==============================] - 24s 646us/sample - loss: 0.5807 - accuracy: 0.7382 - val_loss: 0.5599 - val_accuracy: 0.7551
Epoch 5/10
36529/36529 [==============================] - 24s 646us/sample - loss: 0.5597 - accuracy: 0.7517 - val_loss: 0.5440 - val_accuracy: 0.7719
Epoch 6/10
36529/36529 [==============================] - 23s 640us/sample - loss: 0.5532

In [18]:
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score
predictions = model.predict(x=balancedData_test)
threshold = 0.5
acc_60s =  accuracy_score(predictions>threshold,balancedLabels_test)
recall_60s = recall_score(predictions>threshold,balancedLabels_test)
auc_60s = roc_auc_score(predictions>threshold,balancedLabels_test)
print("Test Accuracy:", acc_60s)
print("Recall Accuracy:", recall_60s)
print("AUC Score:", auc_60s)



Test Accuracy: 0.6607382550335571
Recall Accuracy: 0.7155715571557155
AUC Score: 0.6718574746720257


In [19]:
balancedData.shape

(45662, 900, 6)

# Pytorch Model

## Create Pytorch Dataset

In [5]:
batch_size = 128
torch.manual_seed(random_seed)

In [6]:

train_set_balanced = torch.utils.data.Subset(meal_data_train, train_shuffledUnderSampledBalancedIndices)
test_set_balanced = torch.utils.data.Subset(meal_data_test, test_shuffledUnderSampledBalancedIndices)

train_loader = torch.utils.data.DataLoader(train_set_balanced,batch_size=batch_size, shuffle=True,num_workers=2)
test_loader = torch.utils.data.DataLoader(test_set_balanced ,batch_size=batch_size, shuffle=True,num_workers=2)

## Model 1-SimpleCNNNet

In [7]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.01)
#         nn.init.xavier_uniform_(m.weight.data, gain=nn.init.calculate_gain('relu'))
#         nn.init.normal_(m.weight.data, 0.0, 0.01)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [8]:
from torch import nn
class Discriminator(nn.Module):
    def __init__(self, ngpu, input_shape , out_fea = 1):
        super(Discriminator, self).__init__()
        
        # (input_shape[0], input_shape[1] )=  (number of data point, 6 axis channel )
        in_channels, win_size = input_shape[0], input_shape[1]
        self.in_channels = in_channels
        self.ngpu = ngpu  
        self.out_fea = out_fea
        filter_size = 10
        self.conv0 = nn.Conv1d(in_channels = self.in_channels, 
                               out_channels = filter_size, 
                               kernel_size= 44,  stride= 2, padding=0, bias=False)
        
        self.relu0= nn.ReLU()
        
        self.conv1 = nn.Conv1d(filter_size,filter_size, kernel_size= 20,stride= 2, padding=0, bias=False)
        self.relu1= nn.ReLU()
        
        self.conv2 = nn.Conv1d(filter_size,filter_size, kernel_size= 4, stride= 2, padding=0, bias=False)
        self.relu2= nn.ReLU()
        
#         self.conv3 = nn.Conv1d(filter_size,filter_size, kernel_size= 4, stride= 2, padding=0, bias=False)
#         self.relu3= nn.LeakyReLU(0, inplace=True)
        
        self.avgpool = nn.AvgPool1d(kernel_size=10)
        self.flatten = nn.Flatten()
        self.linear1 = None 
        self.relu4 = nn.ReLU()
        self.linear2 = nn.Linear(in_features=200, out_features=out_fea, bias=True)
        self.softmax = nn.Softmax(dim=out_fea)
        self.sigmoid = nn.Sigmoid()
        
        self.std = 1.
        nn.init.normal_(self.conv0.weight.data, 0.0, self.std)
        nn.init.normal_(self.conv1.weight.data, 0.0, self.std)
        nn.init.normal_(self.conv2.weight.data, 0.0, self.std)
        nn.init.normal_(self.linear2.bias.data, 0.0, self.std)
        
    def l1_loss(self,factor=0.01):
        l1_crit = nn.L1Loss(size_average=False,reduction='sum')
        reg_loss = 0.
        loss = 0.
        layers = [self.conv0, self.conv1, self.conv2]
        for layer in layers:
            for p in layer.parameters():
                #print(p)
                reg_loss += l1_crit(p, torch.zeros(p.shape))

        loss = factor * reg_loss
        return loss

    def forward(self, input):
#         print("input shape:",input.shape)
        x = input.permute(0,2,1)
        x = self.conv0(x)
        x = self.relu0(x)
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.avgpool(x)
        x = self.flatten(x)
        if self.linear1 == None:
            self.linear1 = nn.Linear(in_features=x.shape[1], out_features=200, bias=True)
            nn.init.normal_(self.linear1.weight.data, 0.0, self.std)
            nn.init.normal_(self.linear1.bias.data, 0.0, self.std)
            
        x = self.relu4(self.linear1(x))
        out = self.linear2(x)


        return out


In [9]:
# Test with model 
from torch.nn import functional as F
input_shape = (train_set_balanced[0][0].shape[1], train_set_balanced[0][0].shape[0])
print(input_shape)
model = Discriminator( ngpu=1, input_shape =input_shape , out_fea = 1).to(device)
model.apply(weights_init)
for i, (x, label) in enumerate(train_loader):
    if i ==2:
        x = x.to(device)
        label = label.to(device)
        out = model(x)
#         print(out.dtype, label.dtype)
#         print(out)
        preds = out
        print("pred",preds.shape)
        break

l1_crit = nn.L1Loss(size_average=False,reduction='sum').to(device)
reg_loss = torch.tensor(0).to(device)
loss = torch.tensor(0).to(device)
# for p in model.linear1.parameters():
#     print(p.shape)
#     reg_loss += l1_crit(p, torch.zeros(p.shape).to(device)).to(device)

# factor = 0.0005
# loss += factor * reg_loss
# print("Loss:",loss.item())

NameError: name 'balancedData' is not defined

## Training Function

In [10]:
from tqdm import tqdm
from sklearn.metrics import classification_report,recall_score, precision_score


def eval_model(model,dataloader):
    correct = 0.
    total = 0.
    TP = 0
    FN = 0
    # without update
    with torch.no_grad():
        for samples, labels in dataloader:
            samples = samples.to(device)
            labels = labels.to(device)
            outputs = model(samples).to(device).squeeze()
            #print("Output: ", outputs)
            outputs = torch.round(torch.sigmoid(outputs))
            preds = outputs>=0.5
            preds = preds.to(dtype = torch.float)
            preds.requires_grad = False
#             _,preds = torch.max(outputs,1)
            for i in range(len(preds)):
                if preds[i] == 1 and labels[i] == 1:
                    TP += 1
                if preds[i] == 0 and labels[i] == 1:
                    FN += 1
            correct += torch.sum((preds == labels)).item()
            total += float(len(labels))
        acc =100 * correct/ total
        recall = TP/(TP+FN)
#         print("Evaluation Acc: %.4f %%,  Recall: %.4f "%(acc , recall))
    return acc, recall
            
            
            
            

def train_model(model,dataloader, optimizer, criterion, lrscheduler, n_epochs=20,
                earlystopping=True, patience= 3, checkpoint_name ="checkpoint.pt" ):
    loss_ls = []
    train_acc_ls = []
    valid_acc_ls = []
    valid_acc = 0.0
    loss =0.0
    train_acc = 0.0
    patience_count = 0
    best_val_score = 0.0
    best_model = None
    
    train_dataloader, valid_dataloader = dataloader
    print("Training set batch amounts:", len(train_dataloader))
    print("Test set :", len(valid_dataloader))
    print("Start Training..")
    
    for e in range(n_epochs):
        running_loss = 0.0
        epoch_loss = 0.0
        running_correct = 0.0
        correct_cnt = 0.0
        total_cnt = 0.0
        TP = 0.
        FN = 0.
        model.train()
        for i, (samples, labels) in enumerate(train_dataloader):
            samples = samples.to(device)
            labels = labels.to(device, dtype=torch.float32)
            
            optimizer.zero_grad()
            # reshape samples
            outputs = model(samples).squeeze()

            #print("Output: ", outputs, "label: ", labels)
            
            # Compute loss
            loss = criterion(outputs, labels)
            L1_loss = model.l1_loss(0.01).to(device)
            loss += L1_loss
            loss.backward()
            optimizer.step()
            loss_ls.append(loss)
            
            # prediction
            #_,preds = torch.max(outputs,1)
            outputs = torch.round(torch.sigmoid(outputs))
            preds = outputs>=0.5
            preds = preds.to(dtype = torch.float)
            preds.requires_grad = False
            
            # Compute count of TP, FN
            for j in range(len(preds)):
                if preds[j] == 1. and labels[j] == 1.:
                    TP += 1
                if preds[j] == 0. and labels[j] == 1.:
                    FN += 1
            
            running_loss += loss.item()
            correct_cnt += torch.sum((preds == labels)).item()
            total_cnt += float(len(labels))
            batch_acc = 100. * (preds == labels).sum().item()/ float(len(labels))
            if i %50 ==0:
                #print("===> Batch: %d,  Batch_Loss: %.4f, Train Acc: %.4f %%,  Recall: %.f\n"%(i, loss,batch_acc, recall))
                pass

            
        
        # Compute accuracy and loss of one epoch
        epoch_loss = running_loss / len(train_dataloader)  
        epoch_acc = 100* correct_cnt/ total_cnt  # in percentage
        correct_cnt = 0.0
        total_cnt = 0.0
        train_recall = TP/(TP+FN)
        
        #Validation mode
        model.eval()
        valid_acc, valid_recall= eval_model(model,valid_dataloader)
        valid_acc_ls.append(valid_acc)   
        if e %1==0:
            print("Epoch: %d,  Epoch_Loss: %.4f, Train Acc: %.4f %%, Train Recall: %.4f "%(e, epoch_loss,
                                                                                     epoch_acc,train_recall))
            print("Validation Acc:  %.4f %%,  Validation Recall: %.4f "%(valid_acc, valid_recall))
        
        # Reset train mode
        model.train()
        lrscheduler.step(valid_acc)
        
        
        # If earlystopping is enabled, then save model if performance is improved
        if earlystopping:
            if valid_acc > best_val_score or best_val_score == 0.0:
                best_val_score = valid_acc
                torch.save(model,checkpoint_name)
                patience_count = 0
                print("Checkpoint Saved")
            else:
                if patience_count < patience:
                    patience_count += 1
                else:
                    break
        print("\n")
        
        
            
    # Load best model
    best_model = torch.load(checkpoint_name)
    print("Load Best Model.")
    print("Training completed")
        
    return model, best_model,best_val_score,loss_ls, train_acc_ls, valid_acc_ls
            

In [12]:
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# torch.cuda.set_device(0)
# device ="cpu"

input_shape = (train_set_balanced[0][0].shape[1], train_set_balanced[0][0].shape[0])
model_1 = Discriminator( ngpu=1, input_shape =input_shape , out_fea = 1).to(device)

# criterion = nn.BCELoss()
criterion = nn.BCEWithLogitsLoss()
# optimizer_1 = optim.SGD(model_1.parameters(),lr=0.01,momentum = 0.9,weight_decay=0.01)
optimizer_1 = optim.Adam(model_1.parameters(),lr=0.01)
lrscheduler_1 = optim.lr_scheduler.ReduceLROnPlateau(optimizer_1, mode='min')
dataloader = (train_loader, test_loader)
model_1, best_model_1,val_score,loss_ls, train_acc_ls, valid_acc_ls = train_model(model_1,dataloader, optimizer_1, 
                                                                    criterion, lrscheduler_1, 
                                                                    n_epochs=15, patience = 10,
                                                                    checkpoint_name ="../models/torch_models/checkpoint_model_1.pt" )

Training set batch amounts: 328
Test set : 106
Start Training..


/home/wenkanw/.conda/envs/mlenv/lib/python3.8/site-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Epoch: 0,  Epoch_Loss: 61.8664, Train Acc: 60.3802 %, Train Recall: 0.6082 

Training TP: 12736 ,  FN: 8203 
Validation Acc:  61.2648 %,  Validation Recall: 0.4343 
Checkpoint Saved


Epoch: 1,  Epoch_Loss: 35.0800, Train Acc: 66.2902 %, Train Recall: 0.5215 

Training TP: 10919 ,  FN: 10020 
Validation Acc:  65.4882 %,  Validation Recall: 0.4695 
Checkpoint Saved


Epoch: 2,  Epoch_Loss: 29.3741, Train Acc: 72.3411 %, Train Recall: 0.5974 

Training TP: 12508 ,  FN: 8431 
Validation Acc:  69.7781 %,  Validation Recall: 0.5228 
Checkpoint Saved


Epoch: 3,  Epoch_Loss: 23.4283, Train Acc: 75.9444 %, Train Recall: 0.6472 

Training TP: 13552 ,  FN: 7387 
Validation Acc:  70.9985 %,  Validation Recall: 0.5293 
Checkpoint Saved


Epoch: 4,  Epoch_Loss: 18.0192, Train Acc: 77.3031 %, Train Recall: 0.6687 

Training TP: 14002 ,  FN: 6937 
Validation Acc:  72.0488 %,  Validation Recall: 0.5572 
Checkpoint Saved


Epoch: 5,  Epoch_Loss: 13.4778, Train Acc: 78.3395 %, Train Recall: 0.6944 

Tr

In [ ]:
best_model =best_model_1

In [19]:
torch.manual_seed(random_seed)
input_shape = (train_set_balanced[0][0].shape[1], train_set_balanced[0][0].shape[0])
model_2 = Discriminator( ngpu=1, input_shape =input_shape , out_fea = 1).to(device)

# criterion = nn.BCELoss()
criterion = nn.BCEWithLogitsLoss()
# optimizer_2 = optim.SGD(model_2.parameters(),lr=0.01)
optimizer_2 = optim.Adagrad(model_2.parameters(), lr=0.01, lr_decay=0, weight_decay=0)
# optimizer_2 = optim.Adam(model_2.parameters(),lr=0.01)
lrscheduler_2 = optim.lr_scheduler.ReduceLROnPlateau(optimizer_2, mode='min')
dataloader = (train_loader, test_loader)
model_2, best_model_2,val_score,loss_ls, train_acc_ls, valid_acc_ls = train_model(model_2,dataloader, optimizer_2, 
                                                                    criterion, lrscheduler_2, 
                                                                    n_epochs=15, patience = 10,
                                                                    checkpoint_name ="../models/torch_models/checkpoint_model_2.pt" )

Training set batch amounts: 328
Test set : 106
Start Training..
Epoch: 0,  Epoch_Loss: 132.0078, Train Acc: 58.4125 %, Train Recall: 0.6100 

Training TP: 12772 ,  FN: 8167 
Validation Acc:  59.3935 %,  Validation Recall: 0.6803 
Checkpoint Saved


Epoch: 1,  Epoch_Loss: 68.4900, Train Acc: 65.0413 %, Train Recall: 0.6741 

Training TP: 14114 ,  FN: 6825 
Validation Acc:  62.9364 %,  Validation Recall: 0.7667 
Checkpoint Saved


Epoch: 2,  Epoch_Loss: 58.0614, Train Acc: 68.7927 %, Train Recall: 0.7069 

Training TP: 14801 ,  FN: 6138 
Validation Acc:  65.7396 %,  Validation Recall: 0.7929 
Checkpoint Saved


Epoch: 3,  Epoch_Loss: 53.1808, Train Acc: 70.9919 %, Train Recall: 0.7255 

Training TP: 15192 ,  FN: 5747 
Validation Acc:  69.0459 %,  Validation Recall: 0.5638 
Checkpoint Saved


Epoch: 4,  Epoch_Loss: 50.1739, Train Acc: 72.3124 %, Train Recall: 0.7359 

Training TP: 15408 ,  FN: 5531 
Validation Acc:  70.8136 %,  Validation Recall: 0.6611 
Checkpoint Saved


Epoch: 5,  Epoc

# Debug functions here

In [209]:
a = torch.tensor([[[11,12,13,14],[15,16,17,18],[21,22,23,24]],[[25,26,27,28],[31,32,33,34],[35,36,37,38]]])
a

tensor([[[11, 12, 13, 14],
         [15, 16, 17, 18],
         [21, 22, 23, 24]],

        [[25, 26, 27, 28],
         [31, 32, 33, 34],
         [35, 36, 37, 38]]])

In [210]:
a.shape

torch.Size([2, 3, 4])

In [211]:
a.view(-1, 4,3)

tensor([[[11, 12, 13],
         [14, 15, 16],
         [17, 18, 21],
         [22, 23, 24]],

        [[25, 26, 27],
         [28, 31, 32],
         [33, 34, 35],
         [36, 37, 38]]])

In [221]:
for i in range(len(a)):
    a[i] = a[i].T
a

RuntimeError: The expanded size of the tensor (4) must match the existing size (3) at non-singleton dimension 1.  Target sizes: [3, 4].  Tensor sizes: [4, 3]

In [223]:
a

tensor([[[11, 12, 13, 14],
         [15, 16, 17, 18],
         [21, 22, 23, 24]],

        [[25, 26, 27, 28],
         [31, 32, 33, 34],
         [35, 36, 37, 38]]])

In [231]:
a.permute(0, 2, 1)

tensor([[[11, 15, 21],
         [12, 16, 22],
         [13, 17, 23],
         [14, 18, 24]],

        [[25, 31, 35],
         [26, 32, 36],
         [27, 33, 37],
         [28, 34, 38]]])